# Traffic Sign Detection - Traditional Methods

This notebook implements and evaluates traditional computer vision approaches:
1. HOG + SVM
2. Color + Shape Detection

**Note**: Traditional methods work well on CPU, no GPU needed.

## 1. Setup

In [ ]:
# Setup environment (same as data exploration)
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !git clone https://github.com/YOUR_USERNAME/traffic-sign-detection.git
    %cd traffic-sign-detection
else:
    import os
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')

!pip install -q scikit-learn scikit-image opencv-python matplotlib seaborn joblib tqdm

In [ ]:
import sys
sys.path.append('src')

import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import time

from src.traditional.hog_svm.detector import HOGSVMDetector, SlidingWindowDetector
from src.traditional.color_shape.detector import ColorShapeDetector, SignColor
from src.utils.metrics import DetectionMetrics, SpeedMetrics
from src.utils.visualization import BoundingBoxVisualizer, ResultVisualizer

print("Libraries imported successfully!")

## 2. Load Dataset

In [ ]:
# Load dataset info
import yaml

data_yaml_path = 'data/raw/yolov8/data.yaml'

with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

class_names = data_config['names']
num_classes = data_config['nc']

print(f"Number of classes: {num_classes}")
print(f"Classes: {class_names}")

# Get image paths
data_root = Path('data/raw/yolov8')
train_images = sorted(list((data_root / 'train' / 'images').glob('*.*')))
val_images = sorted(list((data_root / 'valid' / 'images').glob('*.*')))

print(f"\nTrain images: {len(train_images)}")
print(f"Val images: {len(val_images)}")

### 2.1 Helper Functions for Detection Evaluation

In [ ]:
def load_yolo_annotations(label_path, img_width, img_height):
    """
    Load YOLO format annotations and convert to [x1, y1, x2, y2] format
    
    Args:
        label_path: Path to YOLO label file
        img_width: Image width in pixels
        img_height: Image height in pixels
        
    Returns:
        boxes: numpy array of boxes [N, 4] in (x1, y1, x2, y2) format
        classes: numpy array of class IDs [N]
    """
    boxes = []
    classes = []
    
    if not label_path.exists():
        return np.array([]), np.array([])
    
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
                
            class_id = int(parts[0])
            cx, cy, w, h = map(float, parts[1:5])
            
            # Convert from YOLO format (normalized cx, cy, w, h) to pixel coordinates (x1, y1, x2, y2)
            x1 = int((cx - w/2) * img_width)
            y1 = int((cy - h/2) * img_height)
            x2 = int((cx + w/2) * img_width)
            y2 = int((cy + h/2) * img_height)
            
            # Clip to image boundaries
            x1 = max(0, min(x1, img_width - 1))
            y1 = max(0, min(y1, img_height - 1))
            x2 = max(0, min(x2, img_width - 1))
            y2 = max(0, min(y2, img_height - 1))
            
            # Only add valid boxes
            if x2 > x1 and y2 > y1:
                boxes.append([x1, y1, x2, y2])
                classes.append(class_id)
    
    return np.array(boxes), np.array(classes)

print("Helper functions defined successfully!")